In [43]:
import pandas as pd
import os
from os.path import exists
from glob import glob #para eliminar archivos dentro de carpeta
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
#graficos
import plotly.graph_objects as go

In [44]:
try: 
    os.remove(glob('../data/sh_ipc*')[0])
    print('actualizando archivo')
except:
    print('No esta descargado el archivo, no problema')

actualizando archivo


In [45]:
def every_downloads_chrome(driver):
    '''Para ver cuando terminan las descargas'''
    if not driver.current_url.startswith("chrome://downloads"):
        driver.get("chrome://downloads/")
    return driver.execute_script("""
        var items = document.querySelector('downloads-manager')
            .shadowRoot.getElementById('downloadsList').items;
        if (items.every(e => e.state === "COMPLETE"))
            return items.map(e => e.fileUrl || e.file_url);
        """)

In [46]:
#Scrap
indec = 'https://www.indec.gob.ar/indec/web/Nivel4-Tema-3-5-31'
# Descargar los datos de la web

path='../selenium/chromedriver.exe'
carpeta_descarga=os.getcwd().replace('jupyter','data')
#Con getcwd() se encuentra el path absoluto

chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : carpeta_descarga,
        "directory_upgrade": True}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(path,options=chrome_options)

driver.get(indec)

WebDriverWait(driver, 300).until(EC.presence_of_element_located((By.CLASS_NAME, 'a-color2')))
xls_div=driver.find_elements(By.CLASS_NAME, 'a-color2')
hrefs=[]
for element in xls_div:
    elemento=element.get_attribute('href')
    if elemento != None:
        hrefs.append(elemento)
ip_indec = hrefs[1]
driver.get(ip_indec)
paths = WebDriverWait(driver, 300, 1).until(every_downloads_chrome)
driver.quit()


C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_26200\2131741539.py:13: DeprecationWarning:

executable_path has been deprecated, please pass in a Service object



In [47]:
ipc_indec=(pd.read_excel(glob('../data/sh_ipc*')[0],skipfooter=5,header=5,sheet_name=2)
.T.reset_index()
.rename({'index':'Período',3:'inflacion'},axis=1))[['Período','inflacion']][1:].reset_index(drop=True)
ipc_indec.Período=pd.to_datetime(ipc_indec.Período).apply(lambda x: x.strftime('%d/%m/%Y'))
ipc_indec

ipc_geres=(pd.read_excel('../data/inflacion nivel general_ GERES.xlsx',header=2)
.rename({'NIVEL (dic 2001=100)':'inflacion'},axis=1))[['Período','inflacion']]
ipc_geres.Período=pd.to_datetime(ipc_geres.Período).apply(lambda x: x.strftime('%d/%m/%Y'))
#Cambio de base a 2001
coef=ipc_geres[ipc_geres.Período=='01/12/2016'].inflacion.values[0]/ipc_indec.inflacion[0]
ipc_indec.inflacion=ipc_indec.inflacion*coef

ipc_gral=ipc_geres.append(ipc_indec)



C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_26200\703189631.py:14: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [48]:
cotizaciones=pd.read_excel('../output/ITCRM historico.xlsx',sheet_name=2)
tc=cotizaciones[['Período','Estados Unidos']].rename({'Estados Unidos':'dolar'},axis=1)
tc

,Período,dolar
0,21/11/1997,1.00
1,22/11/1997,1.00
2,23/11/1997,1.00
3,24/11/1997,1.00
4,25/11/1997,1.00
...,...,...
9121,11/11/2022,160.70
9122,12/11/2022,160.70
9123,13/11/2022,160.70
9124,14/11/2022,161.78


In [49]:
# ipc_gral['mes']
ipc_gral['mes'] = ipc_gral.Período.apply(lambda x: x[3:5])
ipc_gral['anio'] = ipc_gral.Período.apply(lambda x: x[6:])
tc['mes'] = tc.Período.apply(lambda x: x[3:5])
tc['anio'] = tc.Período.apply(lambda x: x[6:])
tc = tc.groupby(['anio','mes'],as_index=False).mean()

In [50]:
deva_real=ipc_gral.merge(tc,on=['anio','mes'],how='left')
deva_real.inflacion=deva_real.inflacion*(100/deva_real.inflacion.iloc[-1])
deva_real['tc_real_hoy']=deva_real.dolar/deva_real.inflacion*100
deva_real=deva_real.drop_duplicates('Período')

deva_real

,Período,inflacion,mes,anio,dolar,tc_real_hoy
0,01/12/2001,0.502876,12,2001,1.000000,198.8563
1,01/01/2002,0.514405,01,2002,1.600645,311.164581
2,01/02/2002,0.530559,02,2002,2.007143,378.30687
3,01/03/2002,0.551546,03,2002,2.499355,453.154681
4,01/04/2002,0.608844,04,2002,2.950333,484.579743
...,...,...,...,...,...,...
247,01/06/2022,77.089839,06,2022,122.665000,159.119543
248,01/07/2022,82.799264,07,2022,128.403548,155.078127
249,01/08/2022,88.570651,08,2022,135.289677,152.747751
250,01/09/2022,94.031492,09,2022,143.133667,152.218862


In [51]:
def devaluacion_real_plot(df=deva_real):
    deva_real.Período=pd.to_datetime(deva_real.Período,format='%d/%m/%Y')
    x = deva_real["Período"]
    ultimo_mes=deva_real.Período.iloc[-1].strftime('%m/%Y')

    # Datos
    cotizaciones_plot = go.Figure()
    cotizaciones_plot.add_trace(go.Scatter(name=f"TC real", x=x, y=deva_real["tc_real_hoy"], mode="lines",line_width=3))
    # cotizaciones_plot.add_trace(go.Scatter(name=f"China", x=x, y=deva_real["China"], mode="lines"))
    # cotizaciones_plot.add_trace(go.Scatter(name=f"Euro", x=x, y=deva_real["Zona Euro"], mode="lines"))

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True,dtick="M24")
    # cotizaciones_plot.update_yaxes(title_text="Tasa de devaluación")
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
                                    margin ={'b': 70,'l':50,'r':15},
                                    height=600, width=900,
                                    template='none',
                                    title_text=f"Evolución del tipo de cambio (real)<br>A pesos del {ultimo_mes}",
                                    legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.36, orientation='h'))

    note = 'Fuente: BCRA, INDEC, GERES'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.2, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)
    note_2=('El dato de inflación fue calculado como un promedio simple entre los IPC "confiables" (San Luis, CABA, Santa Fe) y los reportados por las consultoras')

    cotizaciones_plot.add_annotation(showarrow=False, text=note_2, font=dict(size=12), xref='paper', x=.949, yref='paper', y=-0.15,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot

devaluacion_real_plot()

In [52]:
devaluacion_real_plot().write_html('../output/devaluacion_argentina.html')

In [53]:
writer = pd.ExcelWriter(f'../output/Tc a precios de hoy.xlsx', engine='xlsxwriter')
deva_real.to_excel(writer, sheet_name='tc', index=False)
writer.save()